In [3]:
import numpy as np
import gurobipy as GRB
import gurobipy as gp
from gurobipy import GRB,quicksum
import math
import time


In [4]:
n=8
size=100

w=[7,11,15,19,23,27,35,42]
b=[52,97,140,191,85,129,26,20]


#K IS STOCK
int(10.69)

10

In [24]:
pi=[0 for i in range(K)]

In [5]:
tic=time.perf_counter()
lags=[]
n=8
size=100
uk=1
dfd=[]
w=[7,11,15,19,23,27,35,42]
b=[52,97,140,191,85,129,26,20]

#Heuristicallly choosing a starting K and UB
#assuming a single stock was cut into the max no of a single size possible
no_in_a_stock=[]
UB=0
for i in w:
    no_in_a_stock.append(int(100/i))
for i in range(len(w)):
    UB+=math.ceil(b[i]/no_in_a_stock[i])
K=UB
K
LB=-1000
tk=1
#initializing pi
pi=[0 for i in range(K)]
iterations=0
df = np.array([[0 for j in range(len(w))] for u in range(K)])
#Subproblem1
#while tk>0.00000001:
for i in range(200): 
    iterations+=1
    #subproblem1
    sub1=gp.Model()
    y=sub1.addVars(K,vtype=GRB.BINARY)

    sub1.setObjective(quicksum(y[k] for k in range(K))-100*quicksum(y[k]*pi[k] for k in range(K)),GRB.MINIMIZE)

    sub1.optimize()
    

    #subproblem2
    sub2=gp.Model()
    x=sub2.addVars(len(w),K,vtype=GRB.INTEGER)
    objective=0
    for i in range(len(w)):
        objective+=w[i]*quicksum(pi[k]*x[i,k]  for k in range(K))    
    sub2.setObjective(objective,GRB.MINIMIZE)
    for i in range(len(w)):
        sub2.addConstr(quicksum(x[i,k] for k in range(K) )==b[i])

    sub2.optimize()
    
    

    lag=sub1.objVal+sub2.objVal
    
    #obtaining the max LB
    if lag>=LB:
        LB=lag
        
#         #obtaining the x and y corresponding to the max LB
#         bestx=[]
#         bestx = np.array([[0 for j in range(len(w))] for u in range(K)])
#         for k in range(K):
#             for i in range(len(w)):
#                 bestx[k][i] = x[i, k].x
#         besty=[]
#         for i in y:
#             besty.append(y[i].x)
#         resultx=bestx
#         resulty=besty
        
        
        
    #extracting the y and x from gurobi into normal lists    

    for k in range(K):
        for i in range(len(w)):
            df[k][i] = x[i, k].x
    
    f=[]
    for i in y:
        f.append(y[i].x)
        
    #obtaining the denominator of tk
    #sd=w[i]*x[i,k]-100*y[k]
    sd=[]
    for k in range(K):
        de=0
        for i in range(len(w)):
            de+=(w[i]*df[k][i]) 
        de=de-100*f[k]
        sd.append(de)        
    bot=np.linalg.norm(sd)**2

    #calculating tk and updating pi
    tk=uk*(UB-LB)/bot
    uk=uk/1.001
    add=[tk*i for i in sd]
    pi=[pi[k] +add[k] for k in range(len(pi))]
    #pi=[pi[j]+[tk*i for i in sd][j] for j in range(len(pi))]
    
    lags.append((sub1.objVal,sub2.objVal,lag,tk))
toc=time.perf_counter()
print('LowerBound' , LB)
print('Runtime ' ,toc-tic)
print('No of iterations', iterations)

Academic license - for non-commercial use only - expires 2021-06-23
Using license file C:\Users\OIBRA\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0xcb19c01c
Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8

  Matrix range     [0e+00, 0e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.6922894

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -0.692289 

Optimal solution found (tolerance 1.00e-04)
Best objective -6.922894406839e-01, best bound -6.922894406839e-01, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0xede9817a
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 161.3842840
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve:


Solution count 1: -1.15471 
No other solutions better than -1.15471

Optimal solution found (tolerance 1.00e-04)
Best objective -1.154712446486e+00, best bound -1.154712446486e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x5270e47a
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 155.6417839
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0

Best objective -1.389516675388e+00, best bound -1.389516675388e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x2535e920
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 149.9292973
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cor

Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0xf20bd050
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 144.2466670
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 0 rows, 166 columns and 0 nonzeros
Model fingerprint: 0x60c1772

...

Variable types: 0 continuous, 166 integer (166 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [8e-05, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -1.2884977

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -1.2885 
No other solutions better than -1.2885

Optimal solution found (tolerance 1.00e-04)
Best objective -1.288497685494e+00, best bound -1.288497685494e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x46ab53f5
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Fou

  Matrix range     [0e+00, 0e+00]
  Objective range  [8e-05, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -1.9193421

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -1.91934 
No other solutions better than -1.91934

Optimal solution found (tolerance 1.00e-04)
Best objective -1.919342057106e+00, best bound -1.919342057106e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x6a40283b
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.1137100
Presolve removed 8 rows and 1328

  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -2.5253495

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -2.52535 
No other solutions better than -2.52535

Optimal solution found (tolerance 1.00e-04)
Best objective -2.525349534056e+00, best bound -2.525349534056e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x50701204
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.1137100
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iter


Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: -3.10404 
No other solutions better than -3.10404

Optimal solution found (tolerance 1.00e-04)
Best objective -3.104039640975e+00, best bound -3.104039640975e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0xbc8b82df
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.1137100
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution


Solution count 1: -3.65575 
No other solutions better than -3.65575

Optimal solution found (tolerance 1.00e-04)
Best objective -3.655754175299e+00, best bound -3.655754175299e+00, gap 0.0000%
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 1328 columns and 1328 nonzeros
Model fingerprint: 0x97a9d30b
Variable types: 0 continuous, 1328 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-02, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 143.1137100
Presolve removed 8 rows and 1328 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 142.548 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.425484764273e+02, best b

In [22]:
print('LowerBound' , LB)
print('Runtime ' ,toc-tic)
print('No of iterations', iterations)

LowerBound 142.3285230140514
Runtime  211.90812259999802
No of iterations 2438


In [6]:
lags

[(0.0, 0.0, 0.0, 8.06353271023534e-07),
 (-0.15695567326456672, 0.0, -0.15695567326456672, 8.055085953188679e-07),
 (-0.3046443198748896, 0.0, -0.3046443198748896, 8.046648063321233e-07),
 (-0.44308408787357756, 0.0, -0.44308408787357756, 8.038219031302784e-07),
 (-0.5722930973909928, 0.0, -0.5722930973909928, 8.029798847812963e-07),
 (-0.6922894406839453, 0.0, -0.6922894406839453, 8.021387503541228e-07),
 (-0.8030911821743343, 0.0, -0.8030911821743343, 8.012984989186863e-07),
 (-0.9047163584877442, 0.0, -0.9047163584877442, 8.004591295458962e-07),
 (-0.9971829784919835, 0.0, -0.9971829784919835, 7.996206413076418e-07),
 (-1.080509023335576, 0.0, -1.080509023335576, 7.987830332767913e-07),
 (-1.1547124464862053, 0.0, -1.1547124464862053, 7.979463045271917e-07),
 (-1.219811173769101, 0.0, -1.219811173769101, 7.971104541336658e-07),
 (-1.2758231034053806, 0.0, -1.2758231034053806, 7.962754811720132e-07),
 (-1.3227661060503437, 0.0, -1.3227661060503437, 7.954413847190083e-07),
 (-1.360658

In [9]:
quicksum(y[k].x for k in range(K))-100*quicksum(y[k].x*pi[k] for k in range(K))

<gurobi.LinExpr: -3.9409971605158844>